In [1]:
from pyspark import SparkContext, SparkConf

conf = (SparkConf()
         .setMaster("local")
         .setAppName("News Processing")
         .set("spark.executor.memory", "9g"))
            
sc = SparkContext(conf = conf)

In [342]:
import pandas as pd
import numpy as np

In [343]:
df_test = pd.read_csv("sampleSubmission.csv", index_col=0)

df_outcomes = pd.read_csv("outcomes.csv", index_col=0)
print df_outcomes.shape
df_outcomes.columns

(619326, 11)


Index([u'is_exciting', u'at_least_1_teacher_referred_donor', u'fully_funded',
       u'at_least_1_green_donation', u'great_chat',
       u'three_or_more_non_teacher_referred_donors',
       u'one_non_teacher_referred_donor_giving_100_plus',
       u'donation_from_thoughtful_donor', u'great_messages_proportion',
       u'teacher_referred_count', u'non_teacher_referred_count'],
      dtype='object')

In [344]:
df_projects = pd.read_csv("projects.csv", index_col=0)
print "Training + test records", df_projects.shape
#leave only projects with outcome
df_projects = df_projects.drop(df_test.index)
print "Training records:", df_projects.shape
df_projects.columns

Training + test records (664098, 34)
Training records: (619326, 34)


Index([u'teacher_acctid', u'schoolid', u'school_ncesid', u'school_latitude',
       u'school_longitude', u'school_city', u'school_state', u'school_zip',
       u'school_metro', u'school_district', u'school_county',
       u'school_charter', u'school_magnet', u'school_year_round',
       u'school_nlns', u'school_kipp', u'school_charter_ready_promise',
       u'teacher_prefix', u'teacher_teach_for_america',
       u'teacher_ny_teaching_fellow', u'primary_focus_subject',
       u'primary_focus_area', u'secondary_focus_subject',
       u'secondary_focus_area', u'resource_type', u'poverty_level',
       u'grade_level', u'fulfillment_labor_materials',
       u'total_price_excluding_optional_support',
       u'total_price_including_optional_support', u'students_reached',
       u'eligible_double_your_impact_match', u'eligible_almost_home_match',
       u'date_posted'],
      dtype='object')

In [345]:
def remove_students_reached_outliers(students):
    if(students < 20000 or pd.isnull(students)):
        return students
    
    #magic, found better value
    return 7500
    
df_projects["new_students_reached"] = df_projects["students_reached"].apply(remove_students_reached_outliers)
df_projects.new_students_reached.fillna(df_projects.new_students_reached.median(), inplace=True) 
df_projects["students_reached"] = df_projects["new_students_reached"]
df_projects.drop('new_students_reached', axis=1, inplace=True)

df_projects.fulfillment_labor_materials.fillna(df_projects.fulfillment_labor_materials.mean(), inplace=True)

df_projects.primary_focus_area.fillna(df_projects.primary_focus_area.value_counts().index[0], inplace=True)
df_projects.primary_focus_subject.fillna(df_projects.primary_focus_subject.value_counts().index[0], inplace=True)
df_projects.grade_level.fillna(df_projects.grade_level.value_counts().index[0], inplace=True)
df_projects.teacher_prefix.fillna(df_projects.teacher_prefix.value_counts().index[0], inplace=True)

df_projects["optional_support"] = df_projects["total_price_including_optional_support"] - df_projects["total_price_excluding_optional_support"]

df_projects.to_csv("training/projects_cleaned.csv")

In [349]:
projects_with_outcomes = pd.concat([df_outcomes, df_projects], axis=1)

In [350]:
df_resources = pd.read_csv("resources.csv", index_col=0)
df_resources = df_resources[~df_resources.projectid.isin(df_test.index)]
print df_resources.shape
df_resources.isnull().sum()

(3413372, 8)


projectid                     0
vendorid                 105016
vendor_name              105043
project_resource_type       194
item_name                 12871
item_number               24017
item_unit_price           13056
item_quantity              7472
dtype: int64

In [351]:
def get_resource_avaraged_data(group):
    """ Return mean and median values for item unit price and item quantity in each group 
    """
    mean_item_unit_price = group["item_unit_price"].mean()
    mean_item_quantity = group["item_quantity"].mean()
    median_item_unit_price = group["item_unit_price"].median()
    median_item_quantity = group["item_quantity"].median()
    items = len(group)
    return pd.Series([mean_item_unit_price, median_item_unit_price, mean_item_quantity, median_item_quantity, items], 
                     index=['mean_item_unit_price', 'median_item_unit_price', 'mean_item_quantity', 'median_item_quantity', 'items'])

resourses_avarage_data = df_resources.groupby(["project_resource_type", "vendor_name"], as_index=False).apply(get_resource_avaraged_data)
resourses_avarage_data = resourses_avarage_data.reset_index()

print len(df_resources)
print df_resources.groupby("project_resource_type", as_index=False).agg({"item_unit_price": [np.mean, len]})
print ''
print df_resources.isnull().sum()

3413372
  project_resource_type item_unit_price         
                                   mean      len
0                 Books      324.876709  1475690
1                 Other      115.581857   245144
2              Supplies       39.077350  1247137
3            Technology      196.762916   438689
4                 Trips       33.701463     5422
5              Visitors       55.828907     1096

projectid                     0
vendorid                 105016
vendor_name              105043
project_resource_type       194
item_name                 12871
item_number               24017
item_unit_price           13056
item_quantity              7472
dtype: int64


In [352]:
def get_resource_data(resourse):
    """ Unpacks resourse items """
    vendor_name = resourse["vendor_name"]
    project_resource_type = resourse["project_resource_type"]
    item_name = resourse["item_name"]
    item_number = resourse["item_number"]
    item_unit_price = resourse["item_unit_price"]
    item_quantity  = resourse["item_quantity"]
    return vendor_name, project_resource_type, item_name, item_number, item_unit_price, item_quantity 

def get_unit_prices(resourse):
    """ Return unit prices for each item. If unit price is present then it simple returns price. 
    Otherwise it return mean median price for item with same vendor name and resource type 
    or with same resource type"""
    vendor_name, project_resource_type, item_name, item_number, item_unit_price, item_quantity  = get_resource_data(resourse)
    if(item_unit_price > 0):
        return item_unit_price
    
    if(pd.notnull(vendor_name) & pd.notnull(project_resource_type)):
        matching_resources = resourses_avarage_data[(resourses_avarage_data.vendor_name == vendor_name) &
                                          (resourses_avarage_data.project_resource_type == project_resource_type)]
        
        if(not matching_resources.empty):
            return matching_resources.iloc[0].median_item_unit_price
        
        return df_resources[df_resources.project_resource_type == project_resource_type].project_resource_type.mean()
        
    return df_resources.item_unit_price.mean()

def get_total_price(resource):
    """ Returns total price for each resource """
    vendor_name, project_resource_type, item_name, item_number, item_unit_price, item_quantity = get_resource_data(resource)
    return item_unit_price * item_quantity

# all missing project_resource_types will be books
df_resources["project_resource_type"].fillna("Books", inplace=True)
# removes skewed data for item unit price
df_resources = df_resources[df_resources["item_unit_price"] < 100000]
# item quantity at least one, should add avaraging based on resource_type and vendor_name combination
df_resources["item_quantity"].fillna(1, inplace=True)
# remove outliers for item_quantity
df_resources = df_resources[df_resources.item_quantity < 20000]
# no point to keep items where we dont know item unit price and item quantity, shoulb be replaced with avaraged price
df_resources = df_resources[(df_resources.item_unit_price > 0) | (df_resources.item_quantity > 0)]

# replaces zero or negative item quantity with quantity of 1 
df_resources["temp_quantity"] = df_resources.item_quantity
df_resources.loc[df_resources.item_quantity <= 0, 'temp_quantity'] = 1
df_resources.item_quantity = df_resources.temp_quantity
df_resources.drop('temp_quantity', axis=1, inplace=True)

df_resources["item_unit_price"] = df_resources.apply(get_unit_prices, axis=1)
# df_resources["total_project_price"] = df_resources.apply(get_total_price, axis=1)

In [179]:
print df_outcomes.shape
print "Exciting:", df_outcomes[df_outcomes.is_exciting == "t"].shape
df_outcomes.isnull().sum()

(619326, 11)
Exciting: (36710, 11)


is_exciting                                            0
at_least_1_teacher_referred_donor                  94398
fully_funded                                           0
at_least_1_green_donation                          94398
great_chat                                             0
three_or_more_non_teacher_referred_donors          94398
one_non_teacher_referred_donor_giving_100_plus     94398
donation_from_thoughtful_donor                     94398
great_messages_proportion                         179839
teacher_referred_count                             94398
non_teacher_referred_count                         94398
dtype: int64

In [187]:
# df_useful_outcomes = df_outcomes[pd.notnull(df_outcomes.at_least_1_green_donation)]
# df_teachers_data = pd.concat([df_useful_outcomes, df_projects], axis=1, join="inner")

def get_teachers_data(group):
    projects_count = len(group)
    exciting_projects = len(group[group["is_exciting"] == "t"])
    exciting_projects_rate = 1.0 * exciting_projects / projects_count
    fully_funded_count = len(group[group["fully_funded"] == "t"])
    fully_funded_rate = 1.0 * fully_funded_count / projects_count
    green_donation_count = len(group[group["at_least_1_green_donation"] == "t"])
    green_donation_rate = 1.0 * green_donation_count / projects_count
    great_chat_count = len(group[group["great_chat"] == "t"])
    great_chat_rate = 1.0 * great_chat_count / projects_count
    teacher_referred_count_mean = group["teacher_referred_count"].mean()
    donation_from_thoughtful_donor_count = len(group["donation_from_thoughtful_donor"] == "t")
    donation_from_thoughtful_donor_rate = 1.0 * donation_from_thoughtful_donor_count / projects_count
    non_teacher_referred_count_mean = group["non_teacher_referred_count"].mean()
    
    return pd.Series([projects_count, exciting_projects, exciting_projects_rate, fully_funded_count, 
                      fully_funded_rate, green_donation_count, green_donation_rate, great_chat_count,
                     great_chat_rate, teacher_referred_count_mean, donation_from_thoughtful_donor_count,
                     donation_from_thoughtful_donor_rate, non_teacher_referred_count_mean], 
                     index=['projects_count', 'exciting_projects', 'exciting_projects_rate', 'fully_funded_count', 
                            'fully_funded_rate', 'green_donation_count', 'green_donation_rate', 'great_chat_count'
                            ,'great_chat_rate', 'teacher_referred_count_mean', 'donation_from_thoughtful_donor_count',
                           'donation_from_thoughtful_donor_rate', 'non_teacher_referred_count_mean'])

teachers = df_teachers_data.groupby('teacher_acctid').apply(get_teachers_data)

# teachers.to_csv("/home/makambi/Desktop/DataAnalysis/kaggle/kdd-cup/teachers.csv")

In [263]:
t = teachers.reset_index()
print "Unique teachers:", t.teacher_acctid.unique().shape[0]
print "Unique teachers with projects:", df_projects.teacher_acctid.unique().shape[0]

Unique teachers: 208147
Unique teachers with projects: 249555


In [292]:
# projects_without_teachers = df
# pp = df_projects.reset_index()
# unknown_teachers_projects = pp[~pp.teacher_acctid.isin(t.teacher_acctid.unique())]
print unknown_teachers_projects.projectid.unique().shape
unknown_teachers_projects[:5]

(47654,)


,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted,optional_support
0,316ed8fb3b81402ff6ac8f721bb31192,42d43fa6f37314365d08692e08680973,c0e6ce89b244764085691a1b8e28cb81,6.362701e+10,36.576340,-119.608713,Selma,CA,93662,NaN,...,highest poverty,Grades 6-8,30,555.81,653.89,32,f,f,2014-05-12,98.08
2,32943bb1063267de6ed19fc0ceb4b9a7,37f85135259ece793213aca9d8765542,665c3613013ba0a66e3a2a26b89f1b68,4.103270e+11,45.166039,-122.414576,Colton,OR,97017,rural,...,high poverty,Grades PreK-2,30,430.89,506.93,17,f,f,2014-05-11,76.04
5,eac7d156205f1333de3887d656f46611,ff064802c18e68db7ddb7ea0bf7732e8,929336e5a242d2e67f1a591196ba7701,4.018700e+10,33.298792,-111.827793,Chandler,AZ,85225,suburban,...,highest poverty,Grades 3-5,30,750.92,883.44,20,f,f,2014-05-11,132.52
6,5a3bfdf2e05781ccd0654dee0d51d1cd,085794a9e315b88cb7aec548831572f5,dc34b021fd177cfdb22f22055ca9d04f,1.006000e+10,32.838778,-85.517527,Lafayette,AL,36862,rural,...,highest poverty,Grades PreK-2,30,2291.48,2695.86,320,f,f,2014-05-11,404.38
9,3118962680bb062323c3566197487315,eb0855cc6ea55d173ca8f8e4fd6a9e18,71b4dee33799c2111773bb4444766bf0,3.402640e+11,39.951767,-75.117188,Camden,NJ,8102,urban,...,highest poverty,Grades PreK-2,30,567.84,668.05,28,f,f,2014-05-11,100.21


In [306]:
# do = df_outcomes.reset_index()
# do[do.projectid.isin(unknown_teachers_projects.projectid.unique())]

(619326, 35)

In [192]:
# df_donations = pd.read_csv("donations.csv", index_col=0)
# df_donations.head()
df_donations.isnull().sum()

projectid                                         0
donor_acctid                                      0
donor_city                                  2005592
donor_state                                  603109
donor_zip                                   1712062
is_teacher_acct                                   0
donation_timestamp                                0
donation_to_project                               0
donation_optional_support                         0
donation_total                                    0
dollar_amount                                     0
donation_included_optional_support                0
payment_method                                    0
payment_included_acct_credit                      0
payment_included_campaign_gift_card               0
payment_included_web_purchased_gift_card          0
payment_was_promo_matched                         0
via_giving_page                                   0
for_honoree                                       0
donation_mes

In [200]:
# df_donations["dollar_amount"].value_counts()
# df_donations[["donation_total", "dollar_amount"]]



,donation_total,dollar_amount
donationid,,
431d720bc3dfd75ae445a5eaa0b0638d,50.00,10_to_100
fcfedba1c8a0ba77d280cace80a909f6,31.56,10_to_100
3fa95d29986aa6f401c6719ced3a3ce7,55.35,10_to_100
020ad6bd5e88a35741d23b5e08f8b8e8,10.00,10_to_100
4b44b03f304d6425ae94446686f93cd6,20.00,10_to_100
9be4b22432cae4c9ad0a8f8b45e23549,5.00,under_10
d638145f90a23325b721813b2f8ee7c4,50.00,10_to_100
b39547f29dfc25fb13c6e9e8d940dc43,250.00,100_and_up
471e89e99b76bc0a84b4b7f2c1d85abe,217.66,100_and_up
